In [1]:
from flask import Flask,redirect,url_for,request,render_template
from flask_cors import CORS,cross_origin

In [2]:
from pymongo import MongoClient

In [3]:
app = Flask(__name__,template_folder='templates 2')
CORS(app,support_credentials=True)  
# Set up MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['ADTProject']


In [4]:
@app.route('/add_cashbook',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_cashbook():
    data=request.form.to_dict()
    collection=db['cashbook']
    collection.insert_one(data)
    return "data saved"

In [5]:
@app.route("/add_cashbook",methods=['GET'])
@cross_origin(supports_credentials=True)
def add_cashbook():
    return render_template('cashbook.html')

In [6]:
@app.route('/view_cashbook',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_data():
    collection=db['cashbook']
    books=collection.find()
    return render_template('cashbookview.html',books=books)

In [7]:
@app.route('/add_supplier',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_supplier():
    data=request.form.to_dict()
    collection=db['supplier']
    collection.insert_one(data)
    return "data saved"

In [8]:
@app.route("/add_supplier",methods=['GET'])
@cross_origin(supports_credentials=True)
def add_supplier():
    return render_template('supplier.html')

In [9]:
@app.route('/view_supplier',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_supplierdata():
    collection=db['supplier']
    books=collection.find()
    return render_template('supplierview.html',books=books)

In [10]:
@app.route('/edit_supplier/<name>', methods=['POST', 'GET'])
@cross_origin(supports_credentials=True)
def edit_supplier(name):
    collection=db['supplier']
    item = collection.find_one({'name': name})
    if request.method == 'GET':
        return render_template('edit_supplier.html',mybooks=item)
    collection.update_one({"name": name},
                  { "$set": {
                             "brand": request.form.get('brand'),
                              "address": request.form.get('address'),
                              "contact": request.form.get('contact'),
                              "outstanding": request.form.get('outstanding'),
                              "bank": request.form.get('bank'),
                              "ifsc": request.form.get('ifsc'),
                              "account": request.form.get('account'),
                             }
                 })

    return 'Updated, success'

In [11]:
@app.route('/add_purchasebill', methods=['GET'])
@cross_origin(supports_credentials=True)
def add_purchasebill():
    collection=db['supplier']
    item=collection.find()
    return render_template('purchasebill.html',books=item)
    

In [12]:
@app.route('/add_purchasebill',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_purchasebill():
    data=request.form.to_dict()
    collection=db['purchasebill']
    collection.insert_one(data)
    if request.form.get("status")=="credit":
        y=db['supplier'].find_one({"brand":request.form.get('brand')})
        x=int(y["outstanding"])
        total=x+int(request.form.get('amount'))
        db['supplier'].update_one({"brand":request.form.get('brand')},
                                 {"$set":{
                                     "outstanding":total} 
                                 })
    return "data saved"

In [13]:
@app.route('/view_purchasebill',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_purchasebill():
    collection=db['purchasebill']
    books=collection.find()
    return render_template('view_purchasebill.html',books=books)

In [14]:
@app.route('/add_product', methods=['GET'])
@cross_origin(supports_credentials=True)
def add_product():
    collection=db['purchasebill']
    item=collection.find()
    return render_template('addproduct.html',books=item)

In [15]:
@app.route('/add_product',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_product():
    data=request.form.to_dict()
    collection=db['product']
    collection.insert_one(data)
    return "data saved"

In [16]:
@app.route('/view_product',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_product():
    collection=db['product']
    books=collection.find()
    return render_template('view_product.html',books=books)

In [17]:
@app.route("/add_customer",methods=['GET'])
@cross_origin(supports_credentials=True)
def add_customer():
    return render_template('addcustomer.html')

In [18]:
@app.route('/add_customer',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_customer():
    data=request.form.to_dict()
    collection=db['customer']
    collection.insert_one(data)
    return "data saved"

In [19]:
@app.route('/view_customer',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_customer():
    collection=db['customer']
    books=collection.find()
    return render_template('view_customer.html',books=books)

In [20]:
@app.route('/add_sales', methods=['GET'])
@cross_origin(supports_credentials=True)
def add_sales():
    collection=db['product']
    books=collection.find()
    design_mrp = dict()
    for record in books:
        if int(record['qty'])>0:
            design_mrp[record['design']]=record['mrp']
    collections=db['customer']
    item1=collections.find()
    return render_template('addsalesbill.html',books=books,banana=design_mrp,apples=item1)

In [21]:
@app.route('/add_sales',methods=['POST'])
@cross_origin(supports_credentials=True)
def save_sales():
    data=request.form.get('amount').split(":")
    amount=int(data[1])
    collection=db['salesbill']
    collection.insert_one({"customer":request.form.get('customer'),"dateofsale":request.form.get('dateofsale'),"amount":amount,"paymenttype":request.form.get('paytype')})
    y=db['customer'].find_one({"cname":request.form.get('customer')})
    x=int(y["points"])
    total=x+(amount*0.01)
    db['customer'].update_one({"cname":request.form.get('customer')},
                                 {"$set":{
                                     "points":total} 
                                 })
    a=db['product'].find_one({"design":data[0]})
    b=int(a["qty"])
    newqty=b-1
    db['product'].update_one({"design":data[0]},{"$set":{"qty":newqty}})
    return "data saved"

In [22]:
@app.route('/view_sales',methods=['GET'])
@cross_origin(supports_credentials=True)
def view_sales():
    collection=db['salesbill']
    books=collection.find()
    return render_template('view_salesbill.html',books=books)

In [23]:
@app.route("/",methods=['GET'])
@cross_origin(supports_credentials=True)
def home_page():
    return render_template('homepage.html')

In [ ]:
if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [11/Oct/2023 22:11:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:16] "GET /add_cashbook HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:29] "GET /add_supplier HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:33] "GET /view_supplier HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:35] "GET /edit_supplier/Padamdeep HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:41] "GET /add_product HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:43] "GET /view_product HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:47] "GET /add_sales HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:49] "GET /add_sales HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:11:55] "GET /view_sales HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:12:51] "GET /add_supplier HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:13:33] "GET /view_supplier HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 22:15:18] "GET /edit_supplier/Padamdeep HTTP/1.1" 200 -
